### General

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

# Here is the path of the root dir of this folder in your google drive
path="/content/drive/My Drive/Project"

import os
import sys
os.chdir(path)
sys.path.append(path)

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import json
import random
import copy
import h5py
import math
from tqdm import tqdm
tqdm.pandas()

In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.0 MB/s eta 0:00:00


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.4 MB/s eta 0:00:00


### Synthetic

#### load model and data

In [ ]:
from torch import cuda
import torch
from transformers import PreTrainedTokenizer, PreTrainedModel
from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer

#this model is for synthetic from serialized triplets sentences to a semantic sentence
device = 'cuda' if cuda.is_available() else 'cpu'
syn_model = torch.load(os.path.join(path, 'model/t5_best_model.pt')).to(device)
syn_tokenizer = T5Tokenizer.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
from transformers import PreTrainedTokenizer, PreTrainedModel

MAX_LEN = 512

def predict(tokenizer: PreTrainedTokenizer, model: PreTrainedModel, text: str, device):
    with torch.no_grad():
        inputs = tokenizer(text, max_length=MAX_LEN, padding=True, return_tensors='pt')
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        ids = ids.to(device)
        mask = mask.to(device)
        generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask,
                max_length=MAX_LEN,
                num_beams=2,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True
                )
        preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
    return preds

In [ ]:
dataset = pd.read_csv(os.path.join(path, 'data/entity_to_subgraph.csv'))

In [ ]:
dataset['subgraph'] = dataset['subgraph'].apply(eval)

#### save result

In [ ]:
sentences = []
for subgraph in dataset['subgraph']:
    sentence = ""
    temp = 0
    for triplet in subgraph:
        if temp == 0:
            sentence = ' '.join(triplet)
            temp = 1
        else:
            sentence += ', '
            sentence += ' '.join(triplet[1:])

    preds = predict(syn_tokenizer, syn_model, sentence, device)[0]
    sentences.append(preds)
dataset['synthetic'] = sentences

In [ ]:
columns_to_drop = ['subgraph']
dataset = dataset.drop(columns=columns_to_drop)

In [ ]:
dataset.to_csv(os.path.join(path,'data/memories/syn_memory.csv'),index_label=False)